# Use Label Studio export file as Dataset


Ensure you have a [Label Studio instance installed, and you are able to upload and label data inside a project](https://labelstud.io/guide/get_started)

In [ ]:
!pip install label-studio

When annotation is completed in Label Studio, you can [export data in JSON format](https://labelstud.io/guide/export)

In [1]:
label_studio_file = '/Users/nik/Downloads/project-218-at-2023-10-20-12-02-521de4a2.json'

Let's create a dataset to feed in Adala agent

In [2]:
from adala.datasets import LabelStudioFileDataset

dataset = LabelStudioFileDataset(label_studio_file=label_studio_file)
dataset.info()

Total Label Studio tasks loaded: 65


Create an agent with a basic classification skill, that accepts `text` column from the tasks payload:

In [3]:
from adala.agents import SingleShotAgent
from adala.skills import ClassificationSkill

agent = SingleShotAgent(
    dataset = dataset,
    skill = ClassificationSkill(
        labels=['Subjective', 'Objective'],
        input_template='Text: {{text}}'
    )
)
agent.greet()

/Users/nik/opt/anaconda3/envs/lc/lib/python3.11/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(



Hi, I am a single shot agent.
I can use a single skill to process a dataset.
For example, you can instruct me to label your dataset.
I have a skill name='' description='' instructions='Label the input text with the following labels: {{labels}}' input_template='Text: {{text}}' output_template="Output: {{select 'predictions' options=labels logprobs='score'}}" prediction_field='predictions' labels=['Subjective', 'Objective']


### Run agent

In [4]:
agent.learn(update_instructions=True)

Stop program system LLM prompt was created by concatenating instructions with text input:

Prediction = LLM(Input, Instructions)

We expect the prediction to be equal to the ground truth.
Your task is to craft a revised concise instruction for the LLM. Follow best practices for LLM prompt engineering.
Include 2-3 examples at the end of your response to demonstrate how the new instruction would be applied.
Use the following format for your examples:
Input: ...
Output: ... user Old instruction: Label the input text with the following labels: {{labels}} 
Errors: 
Input: Text: Also the area where my unit broke).- I'm not too fond of the magnetic strap. 
Prediction: Subjective 
Ground truth: Objective 

Input: Text: Beautiful styling though. 
Prediction: Subjective 
Ground truth: Objective 

Input: Text: 1. long lasting battery (you don't have to recharge it as frequentyly as some of the flip phones)2. 
Prediction: Subjective 
Ground truth: Objective 
 
New instruction: assistant Please categorize the given text as either 'Subjective' (expressing personal feelings, tastes, or opinions) or 'Objective' (based on facts, not influenced by personal feelings or bias).

Examples:
Input: Text: Also the area where my unit broke).- I'm not too fond of the magnetic strap.
Output: Objective

Input: Text: Beautiful styling though.
Output: Objective

Input: Text: 1. long lasting battery (you don't have to recharge it as frequently as some of the flip phones)2.
Output: Objective

Done!


AgentStep(experience=ShortTermMemory(dataset=LabelStudioFileDataset(label_studio_file='/Users/nik/Downloads/project-218-at-2023-10-20-12-02-521de4a2.json', ground_truth_column='ground_truth'), predictions=       predictions                                              score
407783  Subjective  {'Subjective': -0.20141328999999997, 'Objectiv...
407784   Objective  {'Subjective': -0.75759894, 'Objective': -0.63...
407785  Subjective  {'Subjective': -0.07428308999999997, 'Objectiv...
407786  Subjective  {'Subjective': -0.16022399999999995, 'Objectiv...
407787   Objective  {'Subjective': -0.8259393, 'Objective': -0.575...
...            ...                                                ...
407843  Subjective  {'Subjective': -0.005928572999999946, 'Objecti...
407844  Subjective  {'Subjective': -0.05488217200000003, 'Objectiv...
407845  Subjective  {'Subjective': -0.09441284999999998, 'Objectiv...
407846   Objective  {'Subjective': -1.1368709, 'Objective': -0.386...
407847  Subjective  {'Sub

# Advanced usage: connect Agent to a running Label Studio instance

Exporting data from Label Studio can be avoided in certain cases by connecting the Adala agent to a currently running Label Studio instance. 

This method also enables the creation of ground truth data while the agent is in its learning phase.

Let's create a dataset that consumes [tasks from a specific project](https://labelstud.io/guide/setup_project) in Label Studio server provided by URL and API key.

In [7]:
from adala.datasets import LabelStudioDataset

live_dataset = LabelStudioDataset(
    label_studio_url='http://localhost:8080',
    label_studio_api_key='681842051079710a6b0ebce5ec56f746e3400c98',
    label_studio_project_id=218
)

The rest remains the same - simply replace agent's dataset with the newly created one, and agent will catch it up automatically

In [8]:
agent.dataset = live_dataset

Now agent can evoke predictions from the real dataset:

In [10]:
run_2 = agent.run()
print(run_2.experience.predictions)

100%|███████████████████████████████████████████████| 5/5 [00:01<00:00,  3.93it/s]

       predictions                                              score
407783  Subjective  {'Subjective': -0.009167442000000012, 'Objecti...
407784   Objective  {'Subjective': -1.752799, 'Objective': -0.1902...
407785  Subjective  {'Subjective': -0.021852259999999967, 'Objecti...
407786   Objective  {'Subjective': -1.1797419000000002, 'Objective...
407787   Objective  {'Subjective': -2.6367831, 'Objective': -0.074...
...            ...                                                ...
407843  Subjective  {'Subjective': -0.0026350935999999705, 'Object...
407844  Subjective  {'Subjective': -0.16972701000000007, 'Objectiv...
407845  Subjective  {'Subjective': -0.20141328999999997, 'Objectiv...
407846   Objective  {'Subjective': -5.0067153, 'Objective': -0.006...
407847  Subjective  {'Subjective': -0.01507002099999995, 'Objectiv...

[65 rows x 2 columns]
